In [3]:
# used to reload the imported modules on save
%load_ext autoreload
%autoreload 2

import utils as u

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
import os

In [72]:
# constants
grid_size = (8,11)
square_size = 11

# getting the images path
images_path = "../images_and_poses_for_project_assignment/"
images_path = [os.path.join(images_path, imagename) for imagename in os.listdir(images_path) if imagename.endswith(".png")]

## Exercise 1

In [79]:
V = []
all_H = []  # saving the homographies for each image

# getting the homographies
for img in images_path:
    _, H = u.get_homography(img, grid_size, square_size)
    all_H.append(H)
    
    v_12 = u.get_v_vector(H, 1, 2)
    v_11 = u.get_v_vector(H, 1, 1)
    v_22 = u.get_v_vector(H, 2, 2)
    
    V.append(v_12)
    V.append(v_11 - v_22)
    
# computing params
V = np.array(V)
K = u.get_intrinsic(V)

# computing extrinsic for one image
all_R = []
all_t = []

for H in all_H:
    R, t = u.get_extrinsic(K, H)
    all_R.append(R)
    all_t.append(t)

# TODO: make a better log
print("Example params: \n")
print(f"- K -\n{K}\n")
print(f"- R -\n{all_R[0]}\n")
print(f"- t - \n{all_t[0]}\n")


Example params: 

- K -
[[1.01843862e+03 4.23642989e+00 6.40932090e+02]
 [0.00000000e+00 1.01903697e+03 3.68015516e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

- R -
[[ 0.93570323  0.15861044 -0.31342108]
 [ 0.03422032 -0.92443408 -0.36112976]
 [-0.35112453  0.32642588 -0.87042366]]

- t - 
[-40.39962025  54.00745131 397.88786277]



## Exercise 2

In [80]:
# getting the image and extrinsics
img_path = images_path[1]
R1 = all_R[1]
t1 = all_t[1]

# combining R and t
G = np.zeros((3, 4))
G[:, :3] = R1
G[:, 3] = t1

# evaluating the projection matrix
P = K @ G

corners = u.get_corners(img_path, grid_size)

error = 0

for index, corner in enumerate(corners):
    u_coord = corner[0]
    v_coord = corner[1]

    grid_size_cv2 = tuple(reversed(grid_size))
    u_index, v_index = np.unravel_index(index, grid_size_cv2)

    # the coordinates of the corner w.r.t. the reference corner at position (0,0) of the corners array
    x_mm = (u_index) * square_size
    y_mm = (v_index) * square_size

    point_m = np.array([x_mm, y_mm, 0, 1])

    projected_u = (P[0, :].transpose() @ point_m) / (P[2, :].transpose() @ point_m)
    projected_v = (P[1, :].transpose() @ point_m) / (P[2, :].transpose() @ point_m)
    
    error += (projected_u - u_coord)**2 + (projected_v - v_coord)**2

print(f"Error: {error}")

Error: 10.532235026694872
